In [ ]:
from flask import Flask, request, jsonify
import requests
import json
import uuid
import os

COMFY_URL = "http://127.0.0.1:8188"
WORKFLOW_FILE = "pipeline_31_05_2025_Erik.json"  # Path to your saved workflow

app = Flask(__name__)

@app.route('/generate', methods=['POST'])
def generate():
    try:
        # Load base workflow from file
        with open(WORKFLOW_FILE, "r") as f:
            workflow = json.load(f)

        # Optional: Update workflow inputs with request data
        request_data = request.json or {}

        # Example: override prompt/seed
        for node in workflow.get("prompt", {}).values():
            if node.get("class_type") == "KSampler":
                node["inputs"]["seed"] = request_data.get("seed", node["inputs"].get("seed", 1234))
            if "positive" in node["inputs"]:
                node["inputs"]["positive"] = request_data.get("positive", node["inputs"]["positive"])
            if "negative" in node["inputs"]:
                node["inputs"]["negative"] = request_data.get("negative", node["inputs"]["negative"])

        # Send to ComfyUI
        response = requests.post(f"{COMFY_URL}/prompt", json=workflow)
        if response.status_code != 200:
            return jsonify({"error": "Failed to send to ComfyUI", "details": response.text}), 500

        prompt_response = response.json()

        # Get output filename(s)
        output_images = []
        for node_id, outputs in prompt_response.get("outputs", {}).items():
            for image in outputs.get("images", []):
                image_path = image.get("filename", "unknown")
                output_images.append(image_path)

        return jsonify({"status": "success", "images": output_images})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(port=5001, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


An attempt was made to access a socket in a way forbidden by its access permissions


SystemExit: 1